<a href="https://colab.research.google.com/github/Alas-oss/Neural-Network-SMS-Text-Classifier/blob/main/Neural-Network-SMS-Text-Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
%pip install tensorflow-datasets
import tensorflow_datasets as tfds
import tensorflow.keras as keras
%pip install numpy==1.24.3 --quiet
import numpy as np
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 80.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-10-09 20:51:17--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.08s   

2025-10-09 20:51:17 (4.09 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-10-09 20:51:18--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [4]:
names = ["class", "message"]

In [5]:
train_file = pd.read_csv(train_file_path, sep='\t', names=names)
train_file

,class,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [6]:
test_file = pd.read_csv(test_file_path, sep='\t', names=names)
test_file

,class,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [7]:
train_message = train_file["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])
test_message = test_file["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_file['class'].values.tolist()])

In [8]:
vocabulary = {}
for message in train_message:
  for vocab in message.split():
    if vocab not in vocabulary:
      vocabulary[vocab] = 1
    else:
      vocabulary[vocab] += 1

In [9]:
VOCAB_SIZE = len(vocabulary)

longest_message = max(train_message, key=lambda p: len(p.split()))
longest_message_words = longest_message.split()
MAX_LENGTH = len(longest_message_words)

In [10]:
encoded_train_message = [one_hot(x, VOCAB_SIZE) for x in train_message]
padded_train_message = pad_sequences(encoded_train_message, maxlen=MAX_LENGTH, padding='post')
encoded_test_message = [one_hot(x, VOCAB_SIZE) for x in test_message]
padded_test_message = pad_sequences(encoded_test_message, maxlen=MAX_LENGTH, padding='post')

In [13]:
#Building the model
model = Sequential([
    Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=MAX_LENGTH),
    Flatten(),
    Dense(1, activation='sigmoid')
])

#Compiling the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

#Early Stopping callback
early_stop = EarlyStopping(
    monitor='val_accuracy',
    min_delta=1e-4,
    patience=25,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

#Training the model
model.fit(
    padded_train_message,
    train_label,
    validation_data=(padded_test_message, test_label),
    callbacks=[early_stop],
    epochs=1000,
    verbose=2
)

Epoch 1/1000


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


131/131 - 3s - 21ms/step - accuracy: 0.9215 - loss: 0.2238 - val_accuracy: 0.9734 - val_loss: 0.0947
Epoch 2/1000
131/131 - 2s - 17ms/step - accuracy: 0.9866 - loss: 0.0548 - val_accuracy: 0.9820 - val_loss: 0.0564
Epoch 3/1000
131/131 - 2s - 12ms/step - accuracy: 0.9923 - loss: 0.0276 - val_accuracy: 0.9864 - val_loss: 0.0493
Epoch 4/1000
131/131 - 3s - 19ms/step - accuracy: 0.9957 - loss: 0.0142 - val_accuracy: 0.9864 - val_loss: 0.0429
Epoch 5/1000
131/131 - 2s - 12ms/step - accuracy: 0.9978 - loss: 0.0083 - val_accuracy: 0.9856 - val_loss: 0.0427
Epoch 6/1000
131/131 - 3s - 20ms/step - accuracy: 0.9993 - loss: 0.0047 - val_accuracy: 0.9878 - val_loss: 0.0393
Epoch 7/1000
131/131 - 3s - 22ms/step - accuracy: 0.9998 - loss: 0.0037 - val_accuracy: 0.9856 - val_loss: 0.0432
Epoch 8/1000
131/131 - 2s - 19ms/step - accuracy: 0.9998 - loss: 0.0027 - val_accuracy: 0.9856 - val_loss: 0.0441
Epoch 9/1000
131/131 - 2s - 13ms/step - accuracy: 0.9998 - loss: 0.0022 - val_accuracy: 0.9864 - val_

In [14]:
#The function should predict messages based on the model
#Returning a list containing the prediction and label

def predict_message(pred_text):
  class_dict = {
      0: "ham",
      1: "spam",
  }
  encoded_message = [one_hot(pred_text, VOCAB_SIZE)]
  padded_message = pad_sequences(encoded_message, maxlen=MAX_LENGTH, padding="post")
  prediction = [model.predict(padded_message)[0][0], class_dict[np.round(model.predict(padded_message)[0][0])]]
  return (prediction)

#example:
pred_text = "How are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[np.float32(0.006572423), 'ham']


In [15]:
#This cell tests the function and model. It doesn't modify the contents.

def test_predictions():
  test_message = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"]
  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_message, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying!")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
You passed the challenge. Great job!
